In [2]:
import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv
import warnings

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torch

import datasets
import util.misc as utils
from config import cfg
import h5py

/home/kmuvcl/anaconda3/envs/debug/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def eul2rotm_ypr(euler):
    R_x = np.array(
        [
            [1, 0, 0],
            [0, np.cos(euler[0]), -np.sin(euler[0])],
            [0, np.sin(euler[0]), np.cos(euler[0])],
        ],
        dtype=np.float32,
    )

    R_y = np.array(
        [
            [np.cos(euler[1]), 0, np.sin(euler[1])],
            [0, 1, 0],
            [-np.sin(euler[1]), 0, np.cos(euler[1])],
        ],
        dtype=np.float32,
    )

    R_z = np.array(
        [
            [np.cos(euler[2]), -np.sin(euler[2]), 0],
            [np.sin(euler[2]), np.cos(euler[2]), 0],
            [0, 0, 1],
        ],
        dtype=np.float32,
    )

    return np.dot(R_z, np.dot(R_x, R_y))


def create_masks(image):
    masks = torch.zeros((1, height, width), dtype=torch.uint8)
    return masks


def read_line_file(filename, min_line_length=10):
    segs = []  # line segments
    # csv 파일 열어서 Line 정보 가져오기
    with open(filename, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            segs.append([float(row[0]), float(row[1]), float(row[2]), float(row[3])])
    segs = np.array(segs, dtype=np.float32)
    lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=1)
    segs = segs[lengths > min_line_length]
    return segs

def coordinate_yup(segs,org_h,org_w):
    H = np.array([0,org_h,0,org_h])
    segs[:,1] = -segs[:,1]
    segs[:,3] = -segs[:,3]
    # segs = -segs
    return(H+segs)
    

def normalize_segs(segs, pp, rho):
    pp = np.array([pp[0], pp[1], pp[0], pp[1]], dtype=np.float32)
    return (segs - pp)/517.97

def focal_length_normalize(segs):
    segs = segs/517.97
    return segs


def normalize_safe_np(v, axis=-1, eps=1e-6):
    de = LA.norm(v, axis=axis, keepdims=True)
    de = np.maximum(de, eps)
    return v / de


def segs2lines_np(segs):
    ones = np.ones(len(segs))
    ones = np.expand_dims(ones, axis=-1)
    p1 = np.concatenate([segs[:, :2], -ones], axis=-1)
    p2 = np.concatenate([segs[:, 2:], -ones], axis=-1)

    lines = np.cross(p1, p2)

    return normalize_safe_np(lines)



def sample_segs_np(segs, num_sample, use_prob=True):
    num_segs = len(segs)
    sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
    mask = np.zeros([num_sample, 1], dtype=np.float32)
    if num_sample > num_segs:
        sampled_segs[:num_segs] = segs
        mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
    # else:
    #     sampled_segs = segs[:num_sample]
    #     mask[:num_segs] = np.ones([num_sample, 1], dtype=np.float32)
    else:
        lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
        prob = lengths / np.sum(lengths)
        idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
        sampled_segs = segs[idxs]
        mask = np.ones([num_sample, 1], dtype=np.float32)
    return sampled_segs, mask

# def segs_np(segs, num_sample, use_prob=True):
#     num_segs = len(segs)
#     sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
#     mask = np.zeros([num_sample, 1], dtype=np.float32)
#     if num_sample > num_segs:
#         sampled_segs[:num_segs] = segs
#         mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
#     # else:
#     #     sampled_segs = segs[:num_sample]
#     #     mask[:num_segs] = np.ones([num_sample, 1], dtype=np.float32)
#     else:
#         lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
#         prob = lengths / np.sum(lengths)
#         idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
#         sampled_segs = segs[idxs]
#         mask = np.ones([num_sample, 1], dtype=np.float32)
#     return sampled_segs, mask

def straight_line(segs):

    # for i in range(len(segs)):
    #     seg = np.vstack([segs[i,2]-segs[i,0],segs[i,3]-segs[i,1]])
    num_line = segs.shape[0]
    seg = np.zeros([num_line,2])
    for i in range(len(segs)):    
        seg[i,0] = segs[i,2]-segs[i,0]
        seg[i,1] = segs[i,3]-segs[i,1]
    return seg


def sample_vert_segs_np(segs, thresh_theta=22.5):
    lines = segs2lines_np(segs)
    (a, b) = lines[:, 0], lines[:, 1]
    theta = np.arctan2(np.abs(b), np.abs(a))
    thresh_theta = np.radians(thresh_theta)
    return segs[theta < thresh_theta]

def compute_vp_similarity(lines,vp):
    lines = torch.tensor(lines)
    vp = torch.tensor(vp)
    cos_sim = TF.cosine_similarity(lines, vp.unsqueeze(0), dim=-1).abs()
    
    return cos_sim

In [4]:
def load_h5py_to_dict(file_path):
        with h5py.File(file_path, 'r') as f:
            return {key: torch.tensor(f[key][:]) for key in f.keys()}

In [24]:
img = "/home/kmuvcl/dataset/data/scannet-vp/scene0191_00/frame-000000-color.png"
h5_path = "/home/kmuvcl/dataset/data/scannet-vp/scene0191_00/frame-000000-color_line.h5py"
npz_file = "/home/kmuvcl/dataset/data/scannet-vp/scene0191_00/frame-000000-vanish.npz"

In [25]:
with np.load(npz_file) as npz:
    vpts = np.array([npz[d] for d in ["x", "y", "z"]])

In [6]:
img = cv2.imread(img)

In [7]:
dic = load_h5py_to_dict(h5_path)

In [8]:
keylines = dic['klines']
num_segs = keylines.shape[0]

In [9]:
org_segs = np.copy(keylines.reshape(num_segs,-1).numpy())

In [10]:
org_segs

array([[3.83488321e+00, 4.90569949e-01, 6.89272537e+01, 3.44223083e+02],
       [1.39446808e+02, 1.31172743e+01, 1.70797470e+02, 3.04490936e+02],
       [3.50844482e+02, 2.65588409e+02, 3.60161926e+02, 3.10856223e+00],
       [3.55430939e+02, 2.31913452e+02, 3.63810028e+02, 1.85442257e+00],
       [7.35627670e+01, 3.48472443e+02, 3.01543518e+02, 3.33104370e+02],
       [3.56488159e+02, 2.28758957e+02, 3.65046448e+02, 1.20473468e+00],
       [4.35814514e+02, 2.26891937e+02, 4.56036774e+02, 5.50108671e-01],
       [4.34915131e+02, 2.23858047e+02, 4.55562988e+02, 1.18629539e+00],
       [8.54507675e+01, 1.34207550e+02, 1.17784302e+02, 3.41928040e+02],
       [3.47507538e+02, 2.00990768e+02, 3.55703735e+02, 3.62349606e+00],
       [1.50685028e+02, 2.33424168e+01, 3.46913483e+02, 1.53155918e+01],
       [1.45915024e+02, 1.57414503e+01, 1.67187210e+02, 2.10875305e+02],
       [1.53718109e+02, 2.29546146e+01, 3.46309509e+02, 1.52344389e+01],
       [1.46870926e+02, 8.03009605e+00, 3.38089966e

In [11]:
org_segs.shape

(226, 4)

In [12]:
img.shape

(512, 512, 3)

In [23]:
plt.figure(figsize=(5,5))
plt.imshow(img, extent=[0, 512, 512, 0])
for i in range(num_segs):
    plt.plot(
        (org_segs[i, 0], org_segs[i, 2]),
        (org_segs[i, 1], org_segs[i, 3]),
        c="r",
        alpha=1.0,
    )                                 
plt.xlim(0, 512)
plt.ylim(512, 0)
plt.axis('off')
plt.savefig('scannet.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [27]:
vpts[0]

array([-0.99024556,  0.11863917,  0.07306044])

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(img, extent=[0, 512, 512, 0])
for i in range(num_segs):
    plt.plot((0, target_vp1[0]), (0, target_vp1[1]), 'r-', alpha=1.0)
    plt.plot((0, target_vp2[0]), (0, target_vp2[1]), 'g-', alpha=1.0)
    plt.plot((0, target_vp3[0]), (0, target_vp3[1]), 'b-', alpha=1.0)                               
plt.xlim(0, 512)
plt.ylim(512, 0)
plt.axis('off')
plt.savefig('scannet_vp.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')